In [18]:
from skimage.feature import hog
from skimage import img_as_ubyte, color, io, exposure,transform
from skimage.transform import resize
from joblib import load
from matplotlib import patches
import matplotlib.pyplot as plt
import cv2
import numpy as np
import random

In [11]:
import os
os.environ["OPENCV_ENABLE_NONFREE"] = "1"
from google.colab import drive

drive.mount('/content/drive')

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/Face Covering Detection'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)

print(os.listdir(GOOGLE_DRIVE_PATH))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['CV2024_CW_Dataset.zip', 'CV2024_CW_Dataset', 'Wild dataset', 'code.pdf', 'Models', 'Face Covering Detection.ipynb', ' test_function.ipynb']


In [25]:
path = os.path.join(GOOGLE_DRIVE_PATH, 'Personal Dataset')

!cp -r '{path}' .

def MaskDetection(path):

  photos = []

  for image in os.listdir(os.path.join('Personal Dataset')):
    if os.path.isfile(os.path.join('Personal Dataset', image)):
      img = io.imread(os.path.join('Personal Dataset', image))
      photos.append(img)

  while len(photos) > 4:
    del photos[random.randint(0, len(photos)-1)]

  classifier = load(os.path.join(GOOGLE_DRIVE_PATH, 'Models', 'HOG_SVM.joblib'))

  def extract_and_resize(img, face_coord, out_shape):
      x, y, w, h = face_coord
      face = img[y : y + h, x : x + w]
      face_resized = transform.resize(face, out_shape, anti_aliasing=True)
      return face_resized

  for img in photos:
    #select faces ref lab 5
    fig1, ax1 = plt.subplots(figsize=(64, 64))

    img_gray = color.rgb2gray(img)
    img_gray = img_as_ubyte(img_gray)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces_coordinates = face_cascade.detectMultiScale(img_gray, 1.05, 25)

    ax1.imshow(img), ax1.set_axis_off()

    #extract faces
    faces = []
    i=0
    for coord in faces_coordinates:
      faces.append(extract_and_resize(img, coord, out_shape=(128, 128)))

    #extract hogs
    fig2, ax2 = plt.subplots(1,len(faces) * 2, figsize=(64, 1))
    des_list = []
    i=0
    for face in faces:
      HOG_des, HOG_image = hog(face, orientations=8, pixels_per_cell=(6, 6),
                      cells_per_block=(8, 8), visualize=True, channel_axis=2)

      HOG_image_rescaled = exposure.rescale_intensity(HOG_image, in_range=(0, 15))
      des_list.append(HOG_des)

      ax2[i*2].imshow(faces[i])
      ax2[i*2].set_axis_off()

      ax2[(i*2) + 1].set_axis_off()
      ax2[(i*2) + 1].imshow(HOG_image_rescaled, cmap='gray')
      i += 1

    hog_features = np.vstack(des_list)

    #predict faces
    predictions = classifier.predict(hog_features)

    def label_face(face, colour):
      ax1.add_patch(
      patches.Rectangle(xy=(face[0], face[1]), width=face[2], height=face[3],
                        fill=False, color=colour, linewidth=12))

    #label faces
    for face, prediction in zip(faces_coordinates, predictions):
      if prediction == "good mask":
        label_face(face, 'g')
      elif prediction == "bad mask":
        label_face(face, 'orange')
      else:
        label_face(face, 'r')

    fig1.tight_layout
    fig2.tight_layout
    plt.show()

MaskDetection(path)

Output hidden; open in https://colab.research.google.com to view.